In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage import filters, measure, morphology
import math
from skimage.measure import label, regionprops_table
from PIL import Image
import colorsys
from skimage import img_as_ubyte, exposure
from os import listdir
from os.path import isfile, join
import os

In [4]:
path = "/Volumes/Aorta/aorta/segmentations/cellSize/tiles/tilesCellSize/tissueAnalyzer/P03/aorta04"
folders = [x[0] for x in os.walk(path)]
folders = sorted(folders)

In [6]:
df = pd.DataFrame()
for folder in folders:
    df = pd.DataFrame()
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if "seg.npy" in f if "._" not in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        masks = np.load(join(folder, file), allow_pickle=True).item()['masks']
        prop_dict = regionprops_table(masks, properties = ['label','area', 'centroid', 'major_axis_length', 'minor_axis_length', "orientation"])
        df_help = pd.DataFrame(prop_dict)
        df_help['age'] = age
        df_help['aorta'] = aorta
        df_help['file'] = file
        df = pd.concat((df, df_help), axis=0, ignore_index=True)
    if not df.empty:
        df.to_csv(join(folder, "cellArea.csv"))